# Hyperparameter Tuning



In [ ]:
#Make sure this is installed
# !pip install scikit-optimize

In [25]:
#import relevant libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Integer

In [26]:
#read train, validation and test data
df_train = pd.read_csv('train_new.csv')
df_test = pd.read_csv('test_new.csv')
df_val = pd.read_csv('val_new.csv')

In [27]:
x_train = df_train.drop(['Units_sold>1000'], axis = 1)
y_train = df_train['Units_sold>1000']

In [28]:
x_test = df_test.drop(['Units_sold>1000'], axis = 1)
y_test = df_test['Units_sold>1000']

In [29]:
x_val = df_val.drop(['Units_sold>1000'], axis = 1)
y_val = df_val['Units_sold>1000']

## Video 2 : Grid Search

<p style = 'color:green'><b>Run all the cells above before you begin</b><p>

- n-estimator controls the number of decision trees which will be built in sequence in a boosted ensemble model. So, increaing the nummber of estimators will lead to better performance of model, but a very high number will lead to overfitting. Since our goal is to increase the performance of the model on test data, we will check the performance of the model across a list of values - [50, 100, 200, 300, 400].

- Now max_depth is used to set the depth of the tree. Higher the number, more the model learns from training data and hence may result in overfitting. Since our model was slightly overfitting at the end of feature engineering  at a max_depth of 9. Let us set the range between 6 to 14 with a gap of 2.

- The learning_rate ontrols the weightage each model gets for the final prediction. A higher value of learning_rate can lead to overfit models. So far we have been using the default value of 0.1. But we will be trying two other values i.e. 0.2 and 0.3 along with the default value this time. 

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
#define the grid
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': range(6, 13, 2),
    'learning_rate': [0.1, 0.2, 0.3]
}

In [32]:
#define the model instance 
model_GBC = GradientBoostingClassifier(random_state=42)

In [33]:
#define grid search cv
grid_search_cv = GridSearchCV(estimator=model_GBC, param_grid=param_grid, n_jobs=-1, verbose=2,
                      cv=5, scoring='f1')

In [34]:
# fit the grid search model
%%time
grid_search_cv.fit(x_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
CPU times: user 43.2 s, sys: 558 ms, total: 43.7 s
Wall time: 17min 40s


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'max_depth': range(6, 13, 2),
                         'n_estimators': [50, 100, 200, 300, 400]},
             scoring='f1', verbose=2)

In [36]:
# Best performing model
grid_search_cv.best_estimator_

GradientBoostingClassifier(learning_rate=0.3, max_depth=10, n_estimators=400,
                           random_state=42)

In [37]:
# Best performing model
grid_search_cv.best_params_

{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 400}

In [39]:
# Mean cross-validated F1 score of the best estimator
grid_search_cv.best_score_

0.9077340243467941

In [40]:
#predict the dependent values
y_train_grid_search_pred = grid_search_cv.predict(x_train)
y_val_grid_search_pred = grid_search_cv.predict(x_val)

f1_train_grid_search = f1_score(y_train, y_train_grid_search_pred)
f1_val_grid_search = f1_score(y_val, y_val_grid_search_pred)

print("F1 Score on Train data:", f1_train_grid_search)
print("F1 Score on Val data:", f1_val_grid_search)

F1 Score on Train data: 1.0
F1 Score on Val data: 0.9122368146758391


## Video 3 - RandomizedSearchCV
<p style = 'color:green'><b>Run all the cells above before you begin</b><p>


In [41]:
from sklearn.model_selection import RandomizedSearchCV

In [42]:
random_cv = RandomizedSearchCV(estimator=model_GBC, n_iter=10, 
                               param_distributions=param_grid, n_jobs=-1, 
                               cv=5, scoring='f1')

In [43]:
%%time
random_cv.fit(x_train, y_train)

CPU times: user 42.8 s, sys: 230 ms, total: 43 s
Wall time: 3min 11s


RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.2, 0.3],
                                        'max_depth': range(6, 13, 2),
                                        'n_estimators': [50, 100, 200, 300,
                                                         400]},
                   scoring='f1')

In [48]:
#best model estimator as per random search
random_cv.best_estimator_

GradientBoostingClassifier(learning_rate=0.3, max_depth=10, n_estimators=400,
                           random_state=42)

In [49]:
random_cv.best_params_

{'n_estimators': 400, 'max_depth': 10, 'learning_rate': 0.3}

In [50]:
#cross validation score for the best model after random search
random_cv.best_score_

0.9077340243467941

In [47]:
y_train_randomcv_pred = random_cv.predict(x_train)

y_val_randomcv_pred = random_cv.predict(x_val)

f1_train_randomcv = f1_score(y_train, y_train_randomcv_pred)
f1_val_randomcv = f1_score(y_val, y_val_randomcv_pred)

print("F1 Score on Train data:", f1_train_randomcv)
print("F1 Score on Val data:", f1_val_randomcv)

F1 Score on Train data: 1.0
F1 Score on Val data: 0.9122368146758391
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=100; total time=   7.6s
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=300; total time=  26.1s
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=400; total time=  31.7s
[CV] END ...learning_rate=0.1, max_depth=8, n_estimators=300; total time=  33.7s
[CV] END ...learning_rate=0.1, max_depth=8, n_estimators=400; total time=  44.8s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=300; total time=  44.0s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=400; total time=  56.9s
[CV] END ..learning_rate=0.1, max_depth=12, n_estimators=300; total time=  51.7s
[CV] END ..learning_rate=0.1, max_depth=12, n_estimators=400; total time= 1.2min
[CV] END ...learning_rate=0.2, max_depth=6, n_estimators=400; total time=  33.7s
[CV] END ...learning_rate=0.2, max_depth=8, n_estimators=200; total time=  22.4s
[CV] END ...learning_rate=0.2, max_depth

[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=100; total time=   7.5s
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=200; total time=  18.1s
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=400; total time=  31.8s
[CV] END ...learning_rate=0.1, max_depth=8, n_estimators=200; total time=  21.9s
[CV] END ...learning_rate=0.1, max_depth=8, n_estimators=300; total time=  34.4s
[CV] END ...learning_rate=0.1, max_depth=10, n_estimators=50; total time=   7.0s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=100; total time=  14.2s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=200; total time=  28.6s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=300; total time=  44.1s
[CV] END ...learning_rate=0.1, max_depth=12, n_estimators=50; total time=   9.3s
[CV] END ..learning_rate=0.1, max_depth=12, n_estimators=100; total time=  17.8s
[CV] END ..learning_rate=0.1, max_depth=12, n_estimators=200; total time=  34.8s
[CV] END ..learning_rate=0.1

[CV] END ....learning_rate=0.1, max_depth=6, n_estimators=50; total time=   3.6s
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=200; total time=  18.3s
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=400; total time=  31.6s
[CV] END ...learning_rate=0.1, max_depth=8, n_estimators=100; total time=  10.9s
[CV] END ...learning_rate=0.1, max_depth=8, n_estimators=200; total time=  22.5s
[CV] END ...learning_rate=0.1, max_depth=8, n_estimators=400; total time=  44.9s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=200; total time=  28.6s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=300; total time=  44.1s
[CV] END ...learning_rate=0.1, max_depth=12, n_estimators=50; total time=   9.4s
[CV] END ..learning_rate=0.1, max_depth=12, n_estimators=100; total time=  17.7s
[CV] END ..learning_rate=0.1, max_depth=12, n_estimators=200; total time=  34.6s
[CV] END ..learning_rate=0.1, max_depth=12, n_estimators=300; total time=  51.7s
[CV] END ...learning_rate=0.

## Video 5 - Bayesian Optimization
<p style = 'color:green'><b>Run all the cells above before you begin</b><p>


In [9]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Integer

In [12]:
# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    GradientBoostingClassifier(),
    {
        'learning_rate': Real(0.1, 0.3),
        'max_depth': Integer(6, 13),
        'n_estimators': Integer(50, 500),
    },
    n_iter=10,
    random_state=0,
    n_jobs=-1
)

In [13]:
%%time
_ = opt.fit(x_train, y_train)

CPU times: user 30.7 s, sys: 1.78 s, total: 32.4 s
Wall time: 7min 29s


In [15]:
opt.best_params_

OrderedDict([('learning_rate', 0.29342624497686),
             ('max_depth', 11),
             ('n_estimators', 242)])

In [14]:
print(opt.score(x_train, y_train))
print(opt.score(x_val, y_val))

1.0
0.8937850229240958
